In [47]:
import pandas as pd
import pymongo
import pandas as pd
import json
import re
import pymongo
import jieba
import operator
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import stats

In [3]:
card_df=pd.read_excel(r'./reg_card_feature_for_gps.xlsx',encoding='utf-16',index_col=0)
card_df.head()

,加油,超商,交通,電影,旅遊機票飯店,網購
三信銀行-VISA白金卡,0.193548,0.096774,0.161290,0.129032,0.354839,0.064516
上海商銀-TeresaCard,0.171123,0.160428,0.042781,0.160428,0.245989,0.219251
上海商銀-小小兵Bello回饋卡,0.103030,0.133333,0.042424,0.109091,0.360606,0.251515
上海商銀-簡單卡,0.093578,0.097248,0.071560,0.086239,0.445872,0.205505
土地銀行-JCB一卡通聯名晶緻卡,0.142857,0.285714,0.000000,0.000000,0.285714,0.285714


In [93]:
card_df2=pd.read_excel(r'./reg_card_filter_for_gps.xlsx',encoding='utf-16',index_col=0)
card_df2.head()

,加油,超商,交通,電影,旅遊機票飯店,網購
上海商銀-小小兵Bello回饋卡,0.103030,0.133333,0.042424,0.109091,0.360606,0.251515
上海商銀-簡單卡,0.093578,0.097248,0.071560,0.086239,0.445872,0.205505
中國信託-LINE Pay卡,0.127333,0.147000,0.068333,0.118667,0.311667,0.227000
中國信託-Yahoo聯名卡,0.151751,0.124514,0.046693,0.151751,0.272374,0.252918
中國信託-秀泰聯名卡,0.133005,0.064039,0.054187,0.389163,0.211823,0.147783


In [4]:
user_df=pd.read_excel(r'./user_behavior.xlsx',encoding='utf-16',index_col=0)
user_df

,加油,超商,交通高鐵,電影,旅遊,網購
user,,,,,,
user1,0.333,0.167,0.833,0.833,0.500,0.333
user2,0.833,0.500,0.000,0.000,0.000,0.166
user3,0.166,1.000,0.000,0.000,0.666,0.666
user4,0.666,0.667,0.666,0.000,0.666,1.000
user5,0.500,0.000,0.000,0.000,0.000,0.500
user6,0.000,0.834,0.000,0.000,0.000,0.833
user7,1.000,0.334,1.000,1.000,0.666,0.000


In [5]:
user_df2=pd.read_excel(r'./user_behavior2.xlsx',encoding='utf-16',index_col=0)
user_df2

,加油,超商,交通高鐵,電影,旅遊,網購
user,,,,,,
user1,0.111037,0.055685,0.277759,0.277759,0.166722,0.111037
user2,0.555704,0.333556,0.000000,0.000000,0.000000,0.110740
user3,0.066453,0.400320,0.000000,0.000000,0.266613,0.266613
user4,0.181719,0.181992,0.181719,0.000000,0.181719,0.272851
user5,0.500000,0.000000,0.000000,0.000000,0.000000,0.500000
user6,0.000000,0.500300,0.000000,0.000000,0.000000,0.499700
user7,0.250000,0.083500,0.250000,0.250000,0.166500,0.000000


In [6]:
user_df3=pd.read_excel(r'./user_behavior3.xlsx',encoding='utf-16',index_col=0)
user_df3

,加油,超商,交通高鐵,電影,旅遊,網購
user,,,,,,
user1,0.125000,0.083333,0.250000,0.250000,0.166667,0.125000
user2,0.400000,0.266667,0.066667,0.066667,0.066667,0.133333
user3,0.095238,0.333333,0.047619,0.047619,0.238095,0.238095
user4,0.178571,0.178571,0.178571,0.035714,0.178571,0.250000
user5,0.333333,0.083333,0.083333,0.083333,0.083333,0.333333
user6,0.062500,0.375000,0.062500,0.062500,0.062500,0.375000
user7,0.233333,0.100000,0.233333,0.233333,0.166667,0.033333


# 使用pearson correlation計算相似度推薦卡片

In [142]:
def gps_rec(user):
    euc=[]
    for i in card_df2.index:
            euc.append(stats.pearsonr(user_df3.filter(items=[user],axis=0).values[0],card_df2.loc[i].values)[0])
    return card_df2.iloc[pd.Series(euc).sort_values(ascending=False)[:5].index].index

In [143]:
gps_rec('user7')

Index(['中國信託-秀泰聯名卡', '國泰世華-台塑聯名卡', '花旗銀行-新世代花旗饗樂生活悠遊卡', '聯邦銀行-全國加油聯名卡',
       '渣打銀行-現金回饋御璽卡'],
      dtype='object')

In [29]:


#函數找卡名
def rec_gps(user):


    #計算相似度
    x=cosine_similarity(user_df.filter(items=["{}".format(user)],axis=0),card_df)
    # print(x)
    a=list(x)
    b=sorted(a,reverse=True)
    blist=b[0:5]
    blist
    c=[]
    if sum(blist)==0:
        return card_df.index[[12,77,101,121,196]]
    else:
        for i in blist:
            d=a.index(i)
            c.append(d)
        #print(c)
        return list(card_df.index[c])
    


In [43]:
#歐式距離
#dist = np.linalg.norm(user_df.filter(items=['user7'],axis=0)-card_df.loc['永豐銀行-美麗華卡'])
#dist
#np.linalg.norm((user_df.filter(items=['user7'],axis=0)-card_df.loc['永豐銀行-美麗華卡']),axis=0)
#np.linalg.norm(card_df.loc['永豐銀行-美麗華卡'],axis=0)
#np.linalg.norm((user_df.filter(items=['user7'],axis=0)))
euc=[]
for i in card_df.index:
    euc.append(pd.Series(np.linalg.norm((user_df.filter(items=['user7'],axis=0)))-np.linalg.norm(card_df.loc[i],axis=0)))
#np.linalg.norm((user_df.filter(items=['user7'],axis=0)))-np.linalg.norm(card_df.loc['永豐銀行-美麗華卡'],axis=0)
#(user_df.filter(items=['user7'],axis=0)-card_df.loc['永豐銀行-美麗華卡'])/

In [80]:
euc=[]
for i in card_df.index:
        euc.append(stats.pearsonr(user_df.filter(items=['user7'],axis=0).values[0],card_df.loc[i].values)[0])

In [130]:
cos_sim3=cosine_similarity(user_df3.filter(items=['user1'],axis=0),card_df2)
cos_list3=pd.Series(pd.Series(cos_sim3[0])).sort_values(ascending = False)
card_df2.iloc[cos_list3[:30].index]

,加油,超商,交通,電影,旅遊機票飯店,網購
臺灣企銀-悠遊聯名卡,0.158305,0.157331,0.122747,0.151973,0.223088,0.186556
新光銀行-寰宇現金回饋卡,0.186335,0.159420,0.078675,0.192547,0.184265,0.198758
新光銀行-義大世界聯名卡,0.197232,0.152249,0.069204,0.204152,0.197232,0.179931
國泰世華-尊榮御璽卡,0.164740,0.150289,0.092486,0.161850,0.254335,0.176301
中國信託-秀泰聯名卡,0.133005,0.064039,0.054187,0.389163,0.211823,0.147783
彰化銀行-VISA無限卡,0.137681,0.134058,0.123188,0.137681,0.326087,0.141304
玉山銀行-家樂福悠遊聯名卡,0.185185,0.135802,0.061728,0.185185,0.209877,0.222222
新光銀行-日本航空聯名卡,0.189655,0.155172,0.051724,0.189655,0.241379,0.172414
第一銀行-菁英御璽卡,0.203125,0.164062,0.046875,0.203125,0.203125,0.179688
彰化銀行-商旅御璽卡,0.187500,0.179688,0.054688,0.187500,0.210938,0.179688


In [87]:
card_df.iloc[226]

加油        0.454545
超商        0.000000
交通        0.000000
電影        0.272727
旅遊機票飯店    0.272727
網購        0.000000
Name: 第一銀行-速邁樂聯名卡, dtype: float64

In [72]:
stats.pearsonr(np.array(user_df.filter(items=['user7'],axis=0).values[0]),np.array(user_df.filter(items=['user6'],axis=0).values[0]))

(-0.9179910584144713, 0.009812425555301908)

In [76]:
card_df.loc[i].values

array([0., 0., 0., 0., 0., 0.])

In [5]:
cos_sim=cosine_similarity(user_df.filter(items=['user7'],axis=0),card_df)
cos_list=pd.Series(pd.Series(cos_sim[0])).sort_values(ascending = False)
card_df.iloc[cos_list[:5].index]

,加油,超商,交通,電影,旅遊機票飯店,網購
永豐銀行-美麗華卡,0.266667,0.100000,0.100000,0.266667,0.266667,0.000000
華南銀行-大甲媽祖認同卡,0.224000,0.056000,0.168000,0.168000,0.264000,0.120000
臺灣企銀-鈦金商旅卡,0.166667,0.166667,0.166667,0.166667,0.266667,0.066667
玉山銀行-幸運PLUS鈦金卡,0.174194,0.187097,0.122581,0.187097,0.212903,0.116129
華南銀行-櫃買贏家生活卡,0.200000,0.000000,0.200000,0.200000,0.200000,0.200000


In [6]:
cos_sim2=cosine_similarity(user_df2.filter(items=['user7'],axis=0),card_df)
cos_list2=pd.Series(pd.Series(cos_sim2[0])).sort_values(ascending = False)
card_df.iloc[cos_list2[:5].index]

,加油,超商,交通,電影,旅遊機票飯店,網購
永豐銀行-美麗華卡,0.266667,0.100000,0.100000,0.266667,0.266667,0.000000
華南銀行-大甲媽祖認同卡,0.224000,0.056000,0.168000,0.168000,0.264000,0.120000
臺灣企銀-鈦金商旅卡,0.166667,0.166667,0.166667,0.166667,0.266667,0.066667
玉山銀行-幸運PLUS鈦金卡,0.174194,0.187097,0.122581,0.187097,0.212903,0.116129
華南銀行-櫃買贏家生活卡,0.200000,0.000000,0.200000,0.200000,0.200000,0.200000


In [13]:
user_df3

,加油,超商,交通高鐵,電影,旅遊,網購
user,,,,,,
user1,0.125000,0.083333,0.250000,0.250000,0.166667,0.125000
user2,0.400000,0.266667,0.066667,0.066667,0.066667,0.133333
user3,0.095238,0.333333,0.047619,0.047619,0.238095,0.238095
user4,0.178571,0.178571,0.178571,0.035714,0.178571,0.250000
user5,0.333333,0.083333,0.083333,0.083333,0.083333,0.333333
user6,0.062500,0.375000,0.062500,0.062500,0.062500,0.375000
user7,0.233333,0.100000,0.233333,0.233333,0.166667,0.033333


In [120]:
user_df.filter(items=['user7'],axis=0)

,加油,超商,交通高鐵,電影,旅遊,網購
user,,,,,,
user7,1.0,0.334,1.0,1.0,0.666,0.0


In [121]:
cosine_similarity(user_df2.filter(items=['user7'],axis=0),card_df)

array([[0.85174508, 0.71779035, 0.58947197, 0.59084887, 0.44128883,
        0.        , 0.4607653 , 0.        , 0.        , 0.        ,
        0.35322187, 0.        , 0.66846768, 0.76396449, 0.        ,
        0.67655511, 0.58420695, 0.35322187, 0.        , 0.54743312,
        0.46048996, 0.        , 0.        , 0.79508097, 0.58638829,
        0.80158276, 0.61474153, 0.72742561, 0.        , 0.53705087,
        0.68741788, 0.77448229, 0.50922999, 0.35322187, 0.        ,
        0.        , 0.76584442, 0.53725066, 0.        , 0.73488062,
        0.        , 0.52795874, 0.        , 0.67718056, 0.63212545,
        0.        , 0.75363693, 0.80787848, 0.        , 0.        ,
        0.64914732, 0.57598464, 0.        , 0.        , 0.73718204,
        0.        , 0.50492431, 0.77016654, 0.74306913, 0.73390914,
        0.        , 0.24976558, 0.70227377, 0.66039393, 0.79326872,
        0.77928706, 0.83392196, 0.64840124, 0.        , 0.        ,
        0.82438305, 0.66734257, 0.        , 0.80

In [122]:
cosine_similarity(user_df.filter(items=['user7'],axis=0),card_df)

array([[0.85174508, 0.71779035, 0.58947197, 0.59084887, 0.44128883,
        0.        , 0.4607653 , 0.        , 0.        , 0.        ,
        0.35322187, 0.        , 0.66846768, 0.76396449, 0.        ,
        0.67655511, 0.58420695, 0.35322187, 0.        , 0.54743312,
        0.46048996, 0.        , 0.        , 0.79508097, 0.58638829,
        0.80158276, 0.61474153, 0.72742561, 0.        , 0.53705087,
        0.68741788, 0.77448229, 0.50922999, 0.35322187, 0.        ,
        0.        , 0.76584442, 0.53725066, 0.        , 0.73488062,
        0.        , 0.52795874, 0.        , 0.67718056, 0.63212545,
        0.        , 0.75363693, 0.80787848, 0.        , 0.        ,
        0.64914732, 0.57598464, 0.        , 0.        , 0.73718204,
        0.        , 0.50492431, 0.77016654, 0.74306913, 0.73390914,
        0.        , 0.24976558, 0.70227377, 0.66039393, 0.79326872,
        0.77928706, 0.83392196, 0.64840124, 0.        , 0.        ,
        0.82438305, 0.66734257, 0.        , 0.80

In [28]:


#函數找卡名
def recommendations():
    
    #輸入偏好
    a,b,c,d,e,f,g,h,i,j,k=0,0,0,0,0,0,0,0,0,0,0
    a=input('卡活動');b=input('保險');c=input('加油');d=input('行動支付');e=input('超商');
    f=input('交通');g=input('電影');h=input('旅遊機票飯店');i=input('網購');j=input('繳稅繳費');k=input('現金回饋');
    
    list01=[a,b,c,d,e,f,g,h,i,j,k]
    for n,i in enumerate(list01):
        if i =='':
            list01[n]=0
        else:
            list01[n]=int(i)

    #偏好正規化        
    INP=pd.DataFrame(columns=['卡活動','保險','加油','行動支付','超商','交通','電影','旅遊機票飯店','網購','繳稅繳費','現金回饋'])
    list02=[]
    if sum(list01) ==0:
        list02=[0]*len(INP.columns)
        INP.loc[0]=list02

    else:   
        for i in list01:
            l02=i/sum(list01)
            list02.append(l02)
        INP.loc[0]=list02
    #計算相似度
    x=cosine_similarity(card_df, INP)
    # print(x)
    a=list(x)
    b=sorted(a,reverse=True)
    blist=b[0:5]
    blist
    c=[]
    if sum(blist)==0:
        return card_df.index[[12,77,101,121,196]]
    else:
        for i in blist:
            d=a.index(i)
            c.append(d)
        #print(c)
        return list(card_df.index[c])
    


array([[0.80987263, 0.74756991, 0.67038755, 0.65649539, 0.47098374,
        0.        , 0.51763964, 0.        , 0.        , 0.        ,
        0.36394257, 0.        , 0.72967354, 0.78620285, 0.        ,
        0.74037652, 0.62115805, 0.36394257, 0.        , 0.5953097 ,
        0.46097633, 0.        , 0.        , 0.85576804, 0.67558571,
        0.77862945, 0.57033594, 0.73937407, 0.24238575, 0.586931  ,
        0.68608512, 0.78694232, 0.49853903, 0.36394257, 0.        ,
        0.        , 0.60632831, 0.55863455, 0.        , 0.64156753,
        0.        , 0.65914157, 0.        , 0.73883338, 0.69655279,
        0.        , 0.89192942, 0.68184307, 0.        , 0.        ,
        0.72787933, 0.62689626, 0.        , 0.        , 0.73337458,
        0.        , 0.64364392, 0.77425665, 0.77243039, 0.73101544,
        0.        , 0.42873886, 0.75436774, 0.69286475, 0.75476357,
        0.81102159, 0.75407614, 0.7186312 , 0.        , 0.        ,
        0.78448359, 0.75311932, 0.        , 0.77

In [27]:
user_df.filter(items=['user1'],axis=0)

,加油,超商,交通高鐵,電影,旅遊,網購
user,,,,,,
user1,0.333,0.167,0.833,0.833,0.5,0.333
